In [ ]:
!pip install transformers
!pip install evaluate
!pip install accelerate -U
!pip install torchmetrics
!pip install -U "neptune[optuna]"
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.5/481.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 10.2 MB/s eta 0:00:00
  Preparing metad

In [ ]:
import os
import torch
import torch.nn as nn
from transformers import AutoModel,AutoTokenizer
from datasets import load_dataset
import numpy as np
import random
import pandas as pd
from google.colab import userdata
import neptune
from captum.attr import LayerIntegratedGradients
from captum.attr import visualization as viz

## Setting up the GPU or CPU

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NEPTUNE_API_TOKEN"] = userdata.get('NEPTUNE_API_TOKEN')
device = torch.device("cuda:0"  if torch.cuda.is_available() else "cpu")

In [ ]:
# @title Neptuna ID
study_id = "FIN-2958" # @param {type:"string"}
trial_id = "FIN-3021" # @param {type:"string"}
project_name="krishanchavinda.official/Fine-Tuning-DCL-Framework"

###Load the Study Run

In [ ]:
run_study = neptune.init_run(with_id=study_id,project=project_name,mode='read-only')

https://app.neptune.ai/krishanchavinda.official/Fine-Tuning-DCL-Framework/e/FIN-2958


### Load the Best Trail Run

In [ ]:
run_trial = neptune.init_run(with_id=trial_id,project=project_name,mode='read-only')

https://app.neptune.ai/krishanchavinda.official/Fine-Tuning-DCL-Framework/e/FIN-3021


In [ ]:
best_prams=run_trial["parameters"].fetch()

In [ ]:
#@title #Variables
SEED = 1234 # @param {type:"integer"}
PADDING_MAX_LENGTH = 45 # @param {type:"integer"}

## Setting Random Seed for Reproducibility

In [ ]:
def setup_seed(seed:int):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [ ]:
setup_seed(SEED)

In [ ]:
# @title Bert Model
bert_model_name = "bert-base-cased" # @param {type:"string"}

## Loading Test Datasets

In [ ]:
data_files = {"test": "test.csv"}
dataset = load_dataset("krishan-CSE/HatEval-Relabeled", data_files=data_files)

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 2724
    })
})

## Loading the Tokernizer for the Transformer Model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

## Define the Bert Architecture

In [ ]:
class DCLArchitecture(nn.Module):
    def __init__(self,dropout:float,bert_model_name:str='bert-base-cased'):
        super(DCLArchitecture, self).__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        self.dim = 768
        self.dense = nn.Linear(self.dim, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self,batch_tokenized, if_train=False):
        input_ids = batch_tokenized['input_ids']
        attention_mask = batch_tokenized['attention_mask']
        bert_output = self.bert(input_ids, attention_mask=attention_mask, output_hidden_states=True)
        bert_cls_hidden_state = bert_output[1]

        if if_train:
            bert_cls_hidden_state_aug = self.dropout(bert_cls_hidden_state)
            bert_cls_hidden_state = torch.cat((bert_cls_hidden_state, bert_cls_hidden_state_aug), dim=1).reshape(-1, self.dim)
        else:
            bert_cls_hidden_state = self.dropout(bert_cls_hidden_state)

        linear_output = self.dense(bert_cls_hidden_state)
        linear_output = linear_output.squeeze(1)

        return bert_cls_hidden_state, linear_output

###Load the Model

In [ ]:
model = DCLArchitecture(bert_model_name=bert_model_name,dropout=best_prams["DROPOUT"])
model.to(device)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

DCLArchitecture(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

### Load the Best Trail Checkpoint

In [ ]:
def load_checkpoint(run: neptune.Run,check_point_name:str):
    model_ext = run[check_point_name]["model"].fetch_extension()
    run[check_point_name]["model"].download()  # Download the checkpoint
    run.wait()
    # Load the checkpoint
    checkpoint = {
        "model_state_dict":torch.load(f"model.{model_ext}"),
    }
    return checkpoint

In [ ]:
check_point_name="model_checkpoints/"

In [ ]:
checkpoint=load_checkpoint(run_study,check_point_name)

In [ ]:
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [ ]:
model.eval()

DCLArchitecture(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

## Model Interpretability

### Define single element Dataset

### Text Examples

1. Hate Speech

In [ ]:
text_example_1=dataset["test"]["text"][2]
target_example_1=dataset["test"]["labels"][2]

In [ ]:
dataset["test"]["labels"][3]

1

2. Not Hate Speech

In [ ]:
text_example_2=dataset["test"]["text"][1]
target_example_2=dataset["test"]["labels"][1]

### Primary Attribution

In [ ]:
text_example=dataset["test"]["text"][3]
target_example=dataset["test"]["labels"][3]

In [ ]:
def construct_input_ref_pair(text,tokenizer,device):
    ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
    sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
    cls_token_id = tokenizer.cls_token_id
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + text_ids + [sep_token_id]

    # construct reference token ids
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]
    indices = input_ids
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    all_tokens =[token.replace('Ġ', '') if 'Ġ' in token else token for token in all_tokens]
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device),all_tokens

In [ ]:
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

In [ ]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

In [ ]:
input_ids, baseline_input_ids,all_tokens = construct_input_ref_pair(text_example,tokenizer,device)
input_attention_mask = construct_attention_mask(input_ids)

In [ ]:
# Define model output
def model_output(input_ids,attention_mask=None):
  inputs={"input_ids":input_ids,"attention_mask":attention_mask}
  _,linear_output=model(inputs)
  preds = torch.sigmoid(linear_output)
  return preds

# Define model input
model_input = model.bert.embeddings

In [ ]:
pred= model_output(input_ids,attention_mask=input_attention_mask)

In [ ]:
lig = LayerIntegratedGradients(model_output, model_input)

In [ ]:
def interpret_text(text,input_ids,input_attention_mask,baseline_input_ids,all_tokens,pred,true_class,tokenizer,device,lig):
    input_attention_mask = construct_attention_mask(input_ids)
    attributions, delta = lig.attribute(inputs=input_ids,
                                    baselines= baseline_input_ids,
                                    additional_forward_args=(input_attention_mask),
                                    return_convergence_delta=True,
                                    n_steps=100
                                    )
    attributions_sum = summarize_attributions(attributions)

    score_vis = viz.VisualizationDataRecord(
                        word_attributions = attributions_sum,
                        pred_prob = pred,
                        pred_class = round(pred),
                        true_class = true_class,
                        attr_class = round(pred),
                        attr_score = attributions_sum.sum(),
                        raw_input_ids = all_tokens,
                        convergence_score = delta)

    viz.visualize_text([score_vis])

In [ ]:
interpret_text(text_example,input_ids,input_attention_mask,baseline_input_ids,all_tokens,pred.item(),target_example,tokenizer,device,lig)

In [ ]:
run_trial.stop()
run_study.stop()

Shutting down background jobs, please wait a moment...
Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/krishanchavinda.official/Fine-Tuning-DCL-Framework/e/FIN-3021/metadata
Shutting down background jobs, please wait a moment...
Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/krishanchavinda.official/Fine-Tuning-DCL-Framework/e/FIN-2958/metadata
